In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

import catboost

from sklearn.metrics import mean_squared_error as mse

df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

In [ ]:
#calculating CPM
#calculating the value that the Advertisers Bid for the month of June
# CPM(the value which was the winning bid value) = 
#((revenue of the publisher*100)/revenue_share_percentage)/measurable_impressions)*1000

def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)

In [ ]:
test_df = df[df.date >= '2019-06-22'].copy()
train_df = df[df.date < '2019-06-22'].copy()

test_df = test_df[test_df.CPM >= 0]
test_df = test_df[test_df.CPM < np.percentile(test_df.CPM, 95)]

train_df = train_df[train_df.CPM >= 0]
train_df = train_df[train_df.CPM < np.percentile(train_df.CPM, 95)]

In [ ]:
drop_cols = ['date', 'CPM', 'total_revenue', 'integration_type_id', 'revenue_share_percent']
drop_cols += ['line_item_type_id', 'order_id']

cat_cols = [x for x in df.columns.tolist() if x not in drop_cols and x not in ['total_impressions', 'viewable_impressions', 'measurable_impressions']]

In [ ]:
clf = catboost.CatBoostRegressor(
    iterations=300,
    cat_features=cat_cols,
    verbose=None,
    random_state=42,
)

clf.fit(train_df.drop(columns=drop_cols), train_df.CPM, silent=True)

pred = clf.predict(test_df.drop(columns=drop_cols))
pred = [max(x,0) for x in pred]

In [ ]:
mse(test_df.CPM, pred)

In [ ]:
fe = pd.Series(data=clf.feature_importances_, index=clf.feature_names_)

fe.sort_values(ascending=False)